In [ ]:
## Creating Tools 
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun  #When Run query they help to interact with outside world
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [2]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250) #how many character do you want
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [4]:
tools = [wiki, arxiv]

In [6]:
## Custom Tools 
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
loader = WebBaseLoader('https://docs.langchain.com/oss/python/langchain/overview')
docs = loader.load()
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorstore = FAISS.from_documents(documents=documents,embedding=embeddings)
retriever = vectorstore.as_retriever()

C:\Users\naman\AppData\Local\Temp\ipykernel_12972\1857186499.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="mxbai-embed-large")


In [8]:
#To convert this retriever into tools :
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langchain-tool", "Langchain Tool for data retrieval")
retriever_tool.name

'langchain-tool'

In [9]:
tools =[wiki, arxiv, retriever_tool]

In [10]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\naman\\Desktop\\GenAIUdemy\\Langchain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langchain-tool', description='Langchain Tool for data retrieval', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001E6C6F2AE60>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E68

In [ ]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

LLM = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=groq_api_key)

In [ ]:
from langchain import hub # hub has pre written prompts
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages  # .messages is uses to show exchanges between user and AI so far

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [14]:
from langchain.agents import create_openai_tools_agent
agents = create_openai_tools_agent(LLM, tools, prompt)
agents

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [15]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agents, tools=tools, verbose=True) #Verbose true is used to show the interaction happening so far
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [16]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langchain-tool` with `{'query': 'Langsmith'}`


​ Core benefits
Standard model interfaceDifferent providers have unique APIs for interacting with models, including the format of responses. LangChain standardizes how you interact with models so that you can seamlessly swap providers and avoid lock-in.Learn moreEasy to use, highly flexible agentLangChain’s agent abstraction is designed to be easy to get started with, letting you build a simple agent in under 10 lines of code. But it also provides enough flexibility to allow you to do all the context engineering your heart desires.Learn moreBuilt on top of LangGraphLangChain’s agents are built on top of LangGraph. This allows us to take advantage of LangGraph’s durable execution, human-in-the-loop support, persistence, and more.Learn moreDebug with LangSmithGain deep visibility into complex agent behavior with visualization tools that trace execution paths, capture state transitions, and

{'input': 'Tell me about Langsmith',
 'output': 'Based on the provided information, it appears that LangChain and LangSmith are related but distinct concepts. LangChain is a software framework that facilitates the integration of large language models (LLMs) into applications, while LangSmith is a tool for debugging and visualizing agent behavior built on top of LangChain.\n\nThe main differences between LangChain and LangSmith are:\n\n1. Purpose: LangChain is a framework for building agents and applications powered by LLMs, while LangSmith is a tool for debugging and visualizing agent behavior.\n2. Functionality: LangChain provides a pre-built agent architecture and model integrations to help users get started quickly, while LangSmith offers visualization tools for tracing execution paths, capturing state transitions, and providing detailed runtime metrics.\n3. Complexity: LangChain is designed to be easy to use and highly flexible, while LangSmith is a more advanced tool that requires

In [18]:
agent_executor.invoke({"input":"What is Machine Learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks wit
Invoking: `wikipedia` with `{'query': 'Machine Learning'}`
responded: It seems like the summary is incomplete. Let me try another search.



Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks wit
Invoking: `wikipedia` with `{'query': 'Machine learning'}`
responded: It seems like the previous attempt at getting a summary of machine learning from Wikipedia failed. Let's try again.



Page: Machine learning
Summary: Machine learning (ML) is a field o

{'input': 'What is Machine Learning',
 'output': 'The function calls did not provide a clear definition of what machine learning is. However, I was able to obtain information about machine learning from Wikipedia, ArXiv, and LangChain.'}

In [20]:
agent_executor.invoke({'input':'What is 1706.03762 paper all about'})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `arxiv` with `{'query': '1706.03762'}`
responded: It seems the response was cut off. Let me try again for more comprehensive output.



Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `langchain-tool` with `{'query': 'A summary of the 1706.03762 arXiv paper about attention is all you need'}`


​ Core benefits
Standard model interfaceDifferent providers have unique APIs for interacting with models, including the format of responses

{'input': 'What is 1706.03762 paper all about',
 'output': 'The paper "Attention Is All You Need" (1706.03762) proposes a new approach to sequence transduction models that eliminates the need for recurrence and convolution in the encoder and decoder. The model, called the Transformer, uses self-attention mechanisms to weigh the importance of different input elements for each position in the output sequence. This allows the model to attend to all positions in the input sequence simultaneously, rather than sequentially, and has been shown to improve the quality of the generated sequences.\n\nThe Transformer model consists of an encoder and a decoder. The encoder takes in a sequence of tokens and produces a continuous representation of the input sequence. The decoder then takes this representation and generates the output sequence one token at a time.\n\nThe key insight behind the Transformer model is the use of self-attention mechanisms to weigh the importance of different input elements